# 🥉 Bronze — Ingest Real Pushshift JSONL

**Run this first, once.** Reads your 20GB JSONL files, enforces schema, writes partitioned Parquet.

Place all your `.jsonl` files in `data/bronze/raw/` before running.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

spark = (
    SparkSession.builder
    .appName('Reddit-Bronze')
    .master('local[2]')
    .config('spark.driver.memory', '4g')
    .config('spark.sql.shuffle.partitions', '8')
    .config('spark.sql.adaptive.enabled', 'true')
    # Pushshift files are large — increase max partition bytes
    .config('spark.sql.files.maxPartitionBytes', '256m')
    .getOrCreate()
)
spark.sparkContext.setLogLevel('WARN')
print('Spark ready:', spark.version)

26/02/26 17:12:35 WARN Utils: Your hostname, terminator resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/26 17:12:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/26 17:12:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark ready: 3.5.1


In [2]:
# Pushshift schema — covers all fields we actually need
# Any field not in this schema is silently dropped (saves memory)
SCHEMA = StructType([
    StructField('id',                StringType(),  False),
    StructField('subreddit',         StringType(),  False),
    StructField('author',            StringType(),  True),
    StructField('title',             StringType(),  True),
    StructField('selftext',          StringType(),  True),
    StructField('score',             IntegerType(), True),
    StructField('upvote_ratio',      DoubleType(),  True),
    StructField('num_comments',      IntegerType(), True),
    StructField('created_utc',       LongType(),    False),
    StructField('url',               StringType(),  True),
    StructField('domain',            StringType(),  True),
    StructField('is_self',           BooleanType(), True),
    StructField('over_18',           BooleanType(), True),
    StructField('stickied',          BooleanType(), True),
    StructField('locked',            BooleanType(), True),
    StructField('removed_by_category', StringType(), True),
    StructField('link_flair_text',   StringType(),  True),
    StructField('total_awards_received', IntegerType(), True),
])

In [3]:
#RAW_PATH    = '../data/bronze/raw/*.jsonl'
#BRONZE_PATH = '../data/bronze/parquet'
RAW_PATH    = '/mnt/c/Users/gusmc/OneDrive/Desktop/reddit_historical_data/data/bronze/raw/*.jsonl'
BRONZE_PATH = '/mnt/c/Users/gusmc/OneDrive/Desktop/reddit_historical_data/data/bronze/parquet'

raw = (
    spark.read
    .schema(SCHEMA)
    .option('mode', 'PERMISSIVE')
    .json(RAW_PATH)
)

print('Raw count:', raw.count())
raw.printSchema()

[Stage 0:======================================================>(124 + 1) / 125]

Raw count: 15339133
root
 |-- id: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- upvote_ratio: double (nullable = true)
 |-- num_comments: integer (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- url: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- is_self: boolean (nullable = true)
 |-- over_18: boolean (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- locked: boolean (nullable = true)
 |-- removed_by_category: string (nullable = true)
 |-- link_flair_text: string (nullable = true)
 |-- total_awards_received: integer (nullable = true)



In [4]:
# Add partition columns + clean up obvious junk
bronze = (
    raw
    .filter(F.col('id').isNotNull() & F.col('subreddit').isNotNull())
    # Remove deleted/removed posts (text is '[deleted]' or '[removed]')
    .filter(~F.lower(F.col('selftext')).isin('[deleted]', '[removed]'))
    .withColumn('created_ts',  F.to_timestamp(F.from_unixtime('created_utc')))
    .withColumn('year',        F.year('created_ts').cast('string'))
    .withColumn('month',       F.lpad(F.month('created_ts').cast('string'), 2, '0'))
    .withColumn('subreddit',   F.lower(F.trim('subreddit')))
    .withColumn('author',      F.lower(F.trim('author')))
    .withColumn('ingested_at', F.current_timestamp())
    # Combine title + selftext into one searchable text field
    .withColumn('full_text',
        F.concat_ws(' ',
            F.coalesce('title', F.lit('')),
            F.coalesce('selftext', F.lit(''))
        )
    )
)

# Show subreddit distribution
bronze.groupBy('subreddit').count().orderBy(F.desc('count')).show(20)

+--------------------+-------+
|           subreddit|  count|
+--------------------+-------+
|            politics|4036663|
|           worldnews|2114916|
|        conservative| 567287|
|            antiwork| 449106|
|      wallstreetbets| 418606|
|      trueoffmychest| 404177|
|    unpopularopinion| 318253|
|         formuladank| 276386|
|    soccercirclejerk| 233001|
|               aitah| 200916|
|        changemyview| 148832|
|            collapse|  67455|
|       dating_advice|  57739|
|femaledatingstrategy|  48006|
+--------------------+-------+



In [6]:
# Write Bronze — partition by subreddit + year
# For 20GB, this will take 10–20 minutes on local
(
    bronze.write
    .mode('overwrite')
    .partitionBy('subreddit', 'year')
    .parquet(BRONZE_PATH)
)
print('Bronze done ✓')
spark.stop()

Bronze done ✓
